<a href="https://colab.research.google.com/github/fymbc/cs4650/blob/main/PA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Chatbot with Seq2Seq Model, Fine-tuned GPT-2
**CS 4650 "Natural Language Processing" Project 3**  
Georgia Tech, Spring 2025 (Instructor: Weicheng Ma)

**To start, first make a copy of this notebook to your local drive, so you can edit it.**

If you want GPUs (which will improve training speed), you can always change your instance type to GPU by going to Runtime -> Change runtime type -> Hardware accelerator.

## 1. Load and Preprocess Data [5 points]

Neural dialog models are Sequence-to-Sequence (Seq2Seq) models that produce conversational response given the dialog history. State-of-the-art dialog models are built by fine-tuning large language models on millions of multi-turn conversations and direct human evaluation (RLHF). However, in this assignment we will narrow our scope to single turn conversations to make the problem easier.  

In this assignment you will implement:
1. Seq2Seq encoder-decoder model
2. Seq2Seq model with attention mechanism
3. Decoding algorithms. First, a naive greedy decoder, then top-$p$ and beam search decoding.

In [ ]:
!gdown 1qYdSlDJ89AvgozK3V5tik8Op93zPbG6e -O processed_CMDC.pkl

Downloading...
From: https://drive.google.com/uc?id=1qYdSlDJ89AvgozK3V5tik8Op93zPbG6e
To: /content/processed_CMDC.pkl
100% 3.49M/3.49M [00:00<00:00, 260MB/s]


In [ ]:
# ===========================================================================
# Run some setup code for this notebook. Don't modify anything in this cell.
# ===========================================================================

import csv, random, re, os, math, pickle, statistics, tqdm, numpy as np
from io import open
from google.colab import files

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.jit import trace
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# ===========================================================================
# A quick note on CUDA functionality (and `.to(model.device)`):
# CUDA is a parallel GPU platform produced by NVIDIA and is used by most GPU
# libraries in PyTorch. CUDA organizes GPUs into device IDs (i.e., "cuda:X" for GPU #X).
# "device" will tell PyTorch which GPU (or CPU) to place an object in. Since
# collab only uses one GPU, we will use 'cuda' as the device if a GPU is available
# and the CPU if not. You will run into problems if your tensors are on different devices.
# ===========================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Oct 26 03:26:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 1.1 Preparing Data

For the dataset we will be using a small sample of single turn input and response pairs from [Cornell Movie Dialog Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html). We filter conversational pairs with sentences > 10 tokens. To reduce your work, we have already created a sample of tokenized, lowercased single turn conversations from Cornell Movie Dialog Corpus.

In [ ]:
def print_list(l, K=None):
	for i, e in enumerate(l):
		if i == K:
			break
		print(e)
	print()

def load_from_pickle(pickle_file):
	with open(pickle_file, "rb") as pickle_in:
		return pickle.load(pickle_in)

In [ ]:
# Loading the pre-processed conversational exchanges (source-target pairs) from pickle data files
all_conversations = load_from_pickle("processed_CMDC.pkl")

# Extract 100 conversations from the end for evaluation and keep the rest for training
eval_conversations = all_conversations[-100:]
all_conversations = all_conversations[:-100]

# Logging data stats
print(f"Number of Training Conversation Pairs = {len(all_conversations)}")
print(f"Number of Evaluation Conversation Pairs = {len(eval_conversations)}")

Number of Training Conversation Pairs = 53065
Number of Evaluation Conversation Pairs = 100


Let's print a couple of conversations to check if they are loaded properly.

In [ ]:
print_list(all_conversations, 5)

('there .', 'where ?')
('you have my word . as a gentleman', 'you re sweet .')
('hi .', 'looks like things worked out tonight huh ?')
('have fun tonight ?', 'tons')
('well no . . .', 'then that s all you had to say .')



### 1.2 Vocabulary

The words in the sentences need to be converted into integer tokens so that the neural model can operate on them. For this purpose, we will create a vocabulary which will convert the input strings into model recognizable integer tokens.

In [ ]:
# ===========================================================================
# Don't modify anything in this cell.
# ===========================================================================

pad_word = "<pad>"
bos_word = "<s>"
eos_word = "</s>"
unk_word = "<unk>"
pad_id = 0
bos_id = 1
eos_id = 2
unk_id = 3

def normalize_sentence(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

class Vocabulary:
    def __init__(self):
        self.word_to_id = {pad_word: pad_id, bos_word: bos_id, eos_word:eos_id, unk_word: unk_id}
        self.word_count = {}
        self.id_to_word = {pad_id: pad_word, bos_id: bos_word, eos_id: eos_word, unk_id: unk_word}
        self.num_words = 4

    def get_ids_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        sent_ids = [bos_id] + [self.word_to_id[word] if word in self.word_to_id \
                               else unk_id for word in sentence.split()] + \
                               [eos_id]
        return sent_ids

    def tokenized_sentence(self, sentence):
        sent_ids = self.get_ids_from_sentence(sentence)
        return [self.id_to_word[word_id] for word_id in sent_ids]

    def decode_sentence_from_ids(self, sent_ids):
        words = list()
        for i, word_id in enumerate(sent_ids):
            if word_id in [bos_id, eos_id, pad_id]:
                # Skip these words
                continue
            else:
                words.append(self.id_to_word[word_id])
        return ' '.join(words)

    def add_words_from_sentence(self, sentence):
        sentence = normalize_sentence(sentence)
        for word in sentence.split():
            if word not in self.word_to_id:
                # add this word to the vocabulary
                self.word_to_id[word] = self.num_words
                self.id_to_word[self.num_words] = word
                self.word_count[word] = 1
                self.num_words += 1
            else:
                # update the word count
                self.word_count[word] += 1

vocab = Vocabulary()
for src, tgt in all_conversations:
    vocab.add_words_from_sentence(src)
    vocab.add_words_from_sentence(tgt)
print(f"Total words in the vocabulary = {vocab.num_words}")

Total words in the vocabulary = 7727


Let's print top 30 vocab words:

In [ ]:
print_list(sorted(vocab.word_count.items(), key=lambda item: item[1], reverse=True), 30)

('.', 84255)
('?', 36822)
('you', 25093)
('i', 18946)
('what', 10765)
('s', 10089)
('it', 9668)
('!', 8872)
('the', 8011)
('t', 7411)
('to', 6929)
('a', 6582)
('that', 5992)
('no', 4931)
('me', 4839)
('do', 4745)
('is', 4434)
('don', 3577)
('are', 3503)
('he', 3413)
('yes', 3384)
('m', 3382)
('not', 3252)
('we', 3252)
('know', 3171)
('re', 2965)
('your', 2809)
('this', 2726)
('yeah', 2708)
('in', 2678)



Print a couple of sentences to verify that the vocabulary is working as intended.

In [ ]:
for src, tgt in all_conversations[:3]:
    sentence = tgt
    word_tokens = vocab.tokenized_sentence(sentence)

    # Automatically adds bos_id and eos_id before and after sentence ids respectively
    word_ids = vocab.get_ids_from_sentence(sentence)
    print(sentence)
    print(word_tokens)
    print(word_ids)
    print(vocab.decode_sentence_from_ids(word_ids))
    print()

word = "the"
word_id = vocab.word_to_id[word]
print(f"Word = {word}")
print(f"Word ID = {word_id}")
print(f"Word decoded from ID = {vocab.decode_sentence_from_ids([word_id])}")

where ?
['<s>', 'where', '?', '</s>']
[1, 6, 7, 2]
where ?

you re sweet .
['<s>', 'you', 're', 'sweet', '.', '</s>']
[1, 8, 15, 16, 5, 2]
you re sweet .

looks like things worked out tonight huh ?
['<s>', 'looks', 'like', 'things', 'worked', 'out', 'tonight', 'huh', '?', '</s>']
[1, 18, 19, 20, 21, 22, 23, 24, 7, 2]
looks like things worked out tonight huh ?

Word = the
Word ID = 47
Word decoded from ID = the


### 1.3 Dataset Preparation

We will use built-in dataset utilities, `torch.utils.data.Dataset` and `torch.utils.data.DataLoader`, to get batched data readily useful for training.

In [ ]:
class SingleTurnMovieDialog_dataset(Dataset):
    """Single-Turn version of Cornell Movie Dialog Cropus dataset."""

    def __init__(self, conversations, vocab, device):
        """
        Args:
            conversations: list of tuple (src_string, tgt_string)
                         - src_string: String of the source sentence
                         - tgt_string: String of the target sentence
            vocab: Vocabulary object that contains the mapping of
                    words to indices
            device: cpu or cuda
        """
        self.conversations = conversations
        self.vocab = vocab
        self.device = device

        def encode(src, tgt):
            src_ids = self.vocab.get_ids_from_sentence(src)
            tgt_ids = self.vocab.get_ids_from_sentence(tgt)
            return (src_ids, tgt_ids)

        # We will pre-tokenize the conversations and save in id lists for later use
        self.tokenized_conversations = [encode(src, tgt) for src, tgt in self.conversations]

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return {"conv_ids":self.tokenized_conversations[idx], "conv":self.conversations[idx]}

def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of dicts {"conv_ids":(src_ids, tgt_ids), "conv":(src_str, trg_str)}.
            - src_ids: list of src piece ids; variable length.
            - tgt_ids: list of tgt piece ids; variable length.
            - src_str: String of src
            - tgt_str: String of tgt
    Returns: dict { "conv_ids":     (src_ids, tgt_ids),
                    "conv":         (src_str, tgt_str),
                    "conv_tensors": (src_seqs, tgt_seqs)}
            src_seqs: torch tensor of shape (src_padded_length, batch_size).
            trg_seqs: torch tensor of shape (tgt_padded_length, batch_size).
            src_padded_length = length of the longest src sequence from src_ids
            tgt_padded_length = length of the longest tgt sequence from tgt_ids

    Implementation tip: You can use the nn.utils.rnn.pad_sequence utility
    function to combine a list of variable-length sequences with padding.
    """
    # Sort conv_ids based on decreasing order of the src_lengths.
    # This is required for efficient GPU computations.
    src_ids = [torch.LongTensor(e["conv_ids"][0]) for e in data]
    tgt_ids = [torch.LongTensor(e["conv_ids"][1]) for e in data]
    src_str = [e["conv"][0] for e in data]
    tgt_str = [e["conv"][1] for e in data]
    data = list(zip(src_ids, tgt_ids, src_str, tgt_str))
    data.sort(key=lambda x: len(x[0]), reverse=True)
    src_ids, tgt_ids, src_str, tgt_str = zip(*data)

    ### BEGIN YOUR CODE ###

    # Pad the src_ids and tgt_ids using token pad_id to create src_seqs and tgt_seqs

    ### END YOUR CODE ###

    return {"conv_ids":(src_ids, tgt_ids), "conv":(src_str, tgt_str), "conv_tensors":(src_seqs.to(device), tgt_seqs.to(device))}

In [ ]:
# Create the DataLoader for all_conversations
dataset = SingleTurnMovieDialog_dataset(all_conversations, vocab, device)

batch_size = 5

data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

Let's test a batch of data to make sure everything is working as intended

In [ ]:
# Test one batch of training data
first_batch = next(iter(data_loader))
print(f"Testing first training batch of size {len(first_batch['conv'][0])}")
print(f"List of source strings:")
print_list(first_batch["conv"][0])
print(f"Tokenized source ids:")
print_list(first_batch["conv_ids"][0])
print(f"Padded source ids as tensor (shape {first_batch['conv_tensors'][0].size()}):")
print(first_batch["conv_tensors"][0])

Testing first training batch of size 5
List of source strings:
is that why you wanted to see me ?
use the bathroom .
for a consideration .
but dad
jane ?

Tokenized source ids:
tensor([  1, 278,  30,  87,   8, 101,  34, 470,  75,   7,   2])
tensor([   1, 1037,   47,  510,    5,    2])
tensor([   1,   85,   13, 5422,    5,    2])
tensor([  1,  36, 875,   2])
tensor([   1, 3100,    7,    2])

Padded source ids as tensor (shape torch.Size([11, 5])):
tensor([[   1,    1,    1,    1,    1],
        [ 278, 1037,   85,   36, 3100],
        [  30,   47,   13,  875,    7],
        [  87,  510, 5422,    2,    2],
        [   8,    5,    5,    0,    0],
        [ 101,    2,    2,    0,    0],
        [  34,    0,    0,    0,    0],
        [ 470,    0,    0,    0,    0],
        [  75,    0,    0,    0,    0],
        [   7,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]], device='cuda:0')


## 2. Baseline Seq2Seq Model [25 points]

With the training `Dataset` and `DataLoader` ready, we can implement our Seq2Seq baseline model.

The model will consist of
1. Shared embedding layer between encoder and decoder that converts the input sequence of word ids to dense embedding representations
2. Bidirectional GRU encoder that encodes the embedded source sequence into hidden representation
3. GRU decoder that predicts target sequence using final encoder hidden representation

In [ ]:
class Seq2seqBaseline(nn.Module):
    def __init__(self, vocab, emb_dim = 300, hidden_dim = 300, num_layers = 2, dropout=0.1):
        """
        Initialize your model's parameters here. To get started, we suggest
        setting all embedding and hidden dimensions to 300, using encoder and
        decoder GRUs with 2 layers, and using a dropout rate of 0.1.

        Implementation tip: To create a bidirectional GRU, you don't need to
        create two GRU networks. Instead use nn.GRU(..., bidirectional=True).
        """
        super().__init__()

        self.num_words = num_words = vocab.num_words
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        ### BEGIN YOUR CODE ###



        ### END YOUR CODE ###

    def encode(self, source):
        """Encode the source batch using a bidirectional GRU encoder.

        Args:
            source: An integer tensor with shape (max_src_sequence_length,
                batch_size) containing subword indices for the source sentences.

        Returns:
            A tuple with three elements:
                encoder_output: The output hidden representation of the encoder
                    with shape (max_src_sequence_length, batch_size, hidden_size).
                    Can be obtained by adding the hidden representations of both
                    directions of the encoder bidirectional GRU.
                encoder_mask: A boolean tensor with shape (max_src_sequence_length,
                    batch_size) indicating which encoder outputs correspond to padding
                    tokens. Its elements should be True at positions corresponding to
                    padding tokens and False elsewhere.
                encoder_hidden: The final hidden states of the bidirectional GRU
                    (after a suitable projection) that will be used to initialize
                    the decoder. This should be a tensor h_n with shape
                    (num_layers, batch_size, hidden_size). Note that the hidden
                    state returned by the bi-GRU cannot be used directly. Its
                    initial dimension is twice the required size because it
                    contains state from two directions.

        The first two return values are not required for the baseline model and will
        only be used later in the attention model. If desired, they can be replaced
        with None for the initial implementation.

        Implementation tip: consider using packed sequences to more easily work
        with the variable-length sequences represented by the source tensor.
        See https://pytorch.org/docs/stable/nn.html#torch.nn.utils.rnn.PackedSequence.

        https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch

        Implementation tip: there are many simple ways to combine the forward
        and backward portions of the final hidden state, e.g. addition, averaging,
        or a linear transformation of the appropriate size. Any of these
        should let you reach the required performance.
        """
        # Compute a tensor containing the length of each source sequence.
        source_lengths = torch.sum(source != pad_id, axis=0).cpu()

        ### BEGIN YOUR CODE ###

        # Compute the mask first

        # Convert word indexes to embeddings

        # Pack padded batch of sequences for RNN module

        # Forward pass through GRU

        # Unpack padding

        # Sum bidirectional GRU outputs

        ### END YOUR CODE ###

        return outputs, mask, hidden

    def decode(self, decoder_input, last_hidden, encoder_output, encoder_mask):
        """Run the decoder GRU for one decoding step from the last hidden state.

        The third and fourth arguments are not used in the baseline model, but are
        included for compatibility with the attention model in the next section.

        Args:
            decoder_input: An integer tensor with shape (1, batch_size) containing
                the subword indices for the current decoder input.
            last_hidden: A pair of tensors h_{t-1} representing the last hidden
                state of the decoder, each with shape (num_layers, batch_size,
                hidden_size). For the first decoding step the last_hidden will be
                encoder's final hidden representation.
            encoder_output: The output of the encoder with shape
                (max_src_sequence_length, batch_size, hidden_size).
            encoder_mask: The output mask from the encoder with shape
                (max_src_sequence_length, batch_size). Encoder outputs at positions
                with a True value correspond to padding tokens and should be ignored.

        Returns:
            A tuple with three elements:
                logits: A tensor with shape (batch_size,
                    vocab_size) containing unnormalized scores for the next-word
                    predictions at each position.
                decoder_hidden: tensor h_n with the same shape as last_hidden
                    representing the updated decoder state after processing the
                    decoder input.
                attention_weights: This will be implemented later in the attention
                    model, but in order to maintain compatible type signatures, we also
                    include it here. This can be None or any other placeholder value.
        """
        # These arguments are not used in the baseline model.
        del encoder_output
        del encoder_mask

        output, hidden = None, None

        ### BEGIN YOUR CODE ###

        # First process the decoder_input via embedding layer

        # Forward through unidirectional GRU

        # Concatenate weighted context vector and GRU output

        ### END YOUR CODE ###

        return output, hidden, None

    def compute_loss(self, source, target):
        """Run the model on the source and compute the loss on the target.

        Args:
            source: An integer tensor with shape (max_source_sequence_length,
                batch_size) containing subword indices for the source sentences.
            target: An integer tensor with shape (max_target_sequence_length,
                batch_size) containing subword indices for the target sentences.

        Returns:
            A scalar float tensor representing cross-entropy loss on the current batch
            divided by the number of target tokens in the batch.
            Many of the target tokens will be pad tokens. You should mask the loss
            from these tokens using appropriate mask on the target tokens loss.

        Implementation tip: don't feed the target tensor directly to the decoder.
        To see why, note that for a target sequence like <s> A B C </s>, you would
        want to run the decoder on the prefix <s> A B C and have it predict the
        suffix A B C </s>.

        You may run self.encode() on the source only once and decode the target
        one step at a time.
        """

        loss = 0

        ### BEGIN YOUR CODE ###

        # Forward pass through encoder

        # Create initial decoder input (start with SOS tokens for each sentence)

        # Set initial decoder hidden state to the encoder's final hidden state

        # Forward batch of sequences through decoder one time step at a time

            # Teacher forcing: next input is current target

            # Calculate and accumulate loss

        ### END YOUR CODE ###

        return loss

We provide a training loop for training the model. You are welcome to modify the training loop by adjusting the learning rate or changing optmization settings.

**Important:** During our testing we found that training the encoder and decoder with different learning rates is crucial for getting good performance over the small dialog corpus. Specifically, the decoder parameter learning rate should be 5 times the encoder parameter learning rate. Hence, add the encoder parameter variable names in the `encoder_parameter_names` as a list. For example, if encoder is using `self.embedding_layer` and `self.encoder_gru` layer then the `encoder_parameter_names` should be `['embedding_layer', 'encoder_gru']`

In [ ]:
def train(model, data_loader, num_epochs, model_file, learning_rate=0.0001):
    """
    Train the model for given number of epochs and save the trained model in
    the final model_file.
    """
    decoder_learning_ratio = 5.0

    ### BEGIN YOUR CODE ###

    encoder_parameter_names = None # <- Add a list of encoder parameter names here!

    ### END YOUR CODE ###

    encoder_named_params = list(filter(lambda kv: any(key in kv[0] for key in encoder_parameter_names), model.named_parameters()))
    decoder_named_params = list(filter(lambda kv: not any(key in kv[0] for key in encoder_parameter_names), model.named_parameters()))
    encoder_params = [e[1] for e in encoder_named_params]
    decoder_params = [e[1] for e in decoder_named_params]
    optimizer = torch.optim.AdamW([
        {'params': encoder_params},
        {
            'params': decoder_params,
            'lr': learning_rate * decoder_learning_ratio
        }
    ], lr = learning_rate)

    clip = 50.0
    for epoch in tqdm.trange(num_epochs, desc="training", unit="epoch"):
        with tqdm.tqdm(data_loader, desc=f"epoch {epoch + 1}", unit="batch", total=len(data_loader), position=0, leave=True) as batch_iterator:
            model.train()
            total_loss = 0.0
            for i, batch_data in enumerate(batch_iterator, start=1):
                source, target = batch_data["conv_tensors"]
                optimizer.zero_grad()
                loss = model.compute_loss(source, target)
                total_loss += loss.item()
                loss.backward()

                # Gradient clipping before taking the step
                _ = nn.utils.clip_grad_norm_(model.parameters(), clip)
                optimizer.step()

                batch_iterator.set_postfix(mean_loss=total_loss / i, current_loss=loss.item())

    # Save the model after training
    torch.save(model.state_dict(), model_file)

We can now train the baseline model.

A correct implementation should get a average train loss of < 3.00  
The code will automatically save and download the model at the end of training.

In [ ]:
# You are welcome to adjust these parameters based on your model implementation.
num_epochs = 6
batch_size = 64

# Reloading the data_loader to increase batch_size
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
baseline_model = Seq2seqBaseline(vocab).to(device)
train(baseline_model, data_loader, num_epochs, "baseline_model.pt")

# Download the trained model to local for future use
files.download('baseline_model.pt')

training: 100%|██████████| 6/6 [02:41<00:00, 26.99s/epoch]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Reload the model from the model file. Useful when you have already trained and saved the model
baseline_model = Seq2seqBaseline(vocab).to(device)
baseline_model.load_state_dict(torch.load("baseline_model.pt", map_location=device))

<All keys matched successfully>

## 3. Decoding [10 points]

### 3.1 Greedy Search

Our language model training objective only predicts the *next* token. We need to be able to generate entire strings from the model. We'll first define a greedy inference procedure here. Later on, we'll implement beam search.


In [ ]:
def predict_greedy(model, sentence, max_length=100):
    """
    Make predictions for the given input using greedy inference.

    Args:
        model: A sequence-to-sequence model.
        sentence: A input string.
        max_length: The maximum length at which to truncate outputs in order to
            avoid non-terminating inference.

    Returns:
        Model's predicted greedy response for the input, represented as string.
    """

    # You should make only one call to model.encode() at the start of the function,
    # and make only one call to model.decode() per inference step.
    model.eval()

    generation = None

    ### BEGIN YOUR CODE ###

    # Forward input through encoder model

    # Prepare encoder's final hidden layer to be first hidden input to the decoder

    # Initialize decoder input with SOS_token

    # Initialize tensors to append decoded words to

    # Iteratively decode one word token at a time

        # Forward pass through decoder

        # Obtain most likely word token and its softmax score

        # Record token and score

        # Prepare current token to be next decoder input (add a dimension)

    # Return collections of word tokens and scores

    ### END YOUR CODE ###

    return generation

Let's chat interactively with our trained baseline Seq2Seq dialog model and save the generated conversations for submission. We will reuse the conversational inputs while testing Seq2Seq + Attention model.

In [ ]:
# ===========================================================================
# Set up chat interactive. Don't modify anything in this cell.
# ===========================================================================

def chat_with_model(model, mode="greedy"):
    if mode == "beam":
        predict_f = predict_beam
    elif mode == "greedy":
        predict_f = predict_greedy
    elif mode == "top-p":
      predict_f = predict_top_p
    else:
      raise ValueError(mode)
    chat_log = list()
    input_sentence = ''
    while(1):
        # Get input sentence
        input_sentence = input('Input > ')
        # Check if it is quit case
        if input_sentence == 'q' or input_sentence == 'quit': break

        generation = predict_f(model, input_sentence)
        if mode == "beam":
            generation = generation[0]
        print('Greedy Response:', generation)
        print()
        chat_log.append((input_sentence, generation))
    return chat_log

*Note: enter "q" or "quit" to end the interactive chat*

In [ ]:
baseline_chat = chat_with_model(baseline_model)

### 3.2 Top-$p$ Sampling

How exactly to draw samples from a language model distribution is an area of ongoing research, in this section we will implement **nucleus sampling** as originally proposed by Holtzman et al., 2020. This is the standard decoding method for most NLP applications, including industry tools like ChatGPT.

Recall our model predicts the probability distribution $P(x|x_{1:i-1})$ over a vocabulary $V$ tokens. Nucleus sampling will draw from a subset of the vocabulary $V^{(p)} \subset V$ which is the smallest set such that:

$$\sum_{x \in V^{(p)}} P(x | x_{1:i-1}) \geq p$$

i.e., the output vocabulary will include all most probable tokens until the total probability exceeds $p$, all the lower probability tokens will be thrown out. Now that we have a smaller set of outputs, we need to normalize their probabilities. So given the total probability mass of the new set $p' = \sum_{x \in V^{(p)}} P(x | x_{1:i-1})$, our final next token probabilities will be:

$$
P'(x|x_{1:i-1}) =
\begin{cases}
P(x|x_{1:i-1})/p' & \text{if } x \in V^{(p)} \\
0 & \text{otherwise}
\end{cases}$$

Additionally, most sampling methods will modify the probability distribution using **temperature**. The temperature will make make generation more stochastic by flattening the probability distribution. This is done before truncating the distribution using top-$p$. Given a temperature parameter $t$ and output logits $u_{1:|V|}$, we apply temperature by re-computing the last softmax layer as:

$$p(x = V|x_{1:i-1}) = \frac{ \exp(u_l/t) }{ \sum_{l'}\exp(u'_l/t')}$$

**For this section, you will use the output logits of your model, and re-compute the final token probability distribution using temperature, then truncate the distribution using nucleus sampling.**

For more information, I highly recommend reading sec. 3 of the original paper:

[The Curious Case of Neural Text Degeneration](https://openreview.net/forum?id=rygGQyrFvH) (Holtzman et al., ICLR 2020)

In [ ]:
def predict_top_p(model, sentence, temperature=0.9, top_p=0.9, max_length=100):
    """
    Make predictions for the given input using top-p sampling.

    First, you will use temperature to re-compute the softmax by re-shaping
    the distribution of logits. Then, you will calculate the candidates whose total
    probability mass is >= p. Lastly, you will normalize and sample from your
    truncated distribution.

    Hint: you can re-use your greedy search implementation for this question. Only the
    area under "TOP-P IMPLEMENTATION" will be different.

    Hint: you may find torch.multinomial() helpful for sampling from the probability distribution
    """
    model.eval()
    generation = None

    ### BEGIN YOUR CODE ###

    # Forward input through encoder model

    # Prepare encoder's final hidden layer to be first hidden input to the decoder

    # Initialize decoder input with SOS_token

    # Initialize tensors to append decoded words to

    # Iteratively decode one word token at a time

        # Forward pass through decoder

        ### BEGIN TOP-P IMPLEMENTATION ###

        # [~2 lines] Apply temperature scaling to the logits to get token probabilities

        # [~1 line] Sort logits in descending order (most to least probable)

        # [~1 line] Calculate the cumulative sum of the token probabilities

        # [~1 line] Find the index where cumulative probability crosses top-p threshold

        # [~2 line] Set the probabilities of all tokens after the top-p threshold to -inf

        # [~4 lines] Re-compute the softmax of token probabilities and sample from the remaining logits
        # if all tokens are under the cutoff, return the most probable token from the original outputs (i.e. degenerate
        # to greedy search)


        ### END TOP-P IMPLEMENTATION ###

        # Record token and score

        # Prepare current token to be next decoder input (add a dimension)

    # Return collections of word tokens and scores

    ### END YOUR CODE ###

    return generation

In [ ]:
# baseline_chat = chat_with_model(baseline_model, mode='top-p')

Great! Let's play around with our baseline model at different parameters. Feel free to change the prompt or parameters.

In [ ]:
PROMPT = 'What is your name?'

print(f'Greedy decoding:\t{predict_greedy(baseline_model, PROMPT)}\n')

for t in [0.00001, 0.1, 0.5, 0.9, 1.5]:
  for _ in range(5):
    generation = predict_top_p(baseline_model, PROMPT, temperature=t, top_p=1)
    print(f'Temperature {t}:\t{generation}')
  print()

How does changing the temperature impact generation? How does this compare to greedy decoding? Feel free to take a look at the nucleus sampling paper (from above) for inspiration into why this may occur.

ANSWER: [**Report your findings here.**]

Now, let's keep the temperature fairly high ($\tau=1$) and sample at different values for $p$.

In [ ]:
for p in [1, 0.9, 0.8, 0.5, 0.2]:
  for _ in range(10):
    generation = predict_top_p(baseline_model, PROMPT, temperature=1, top_p=p)
    print(f'Top-p {p}:\t{generation}')
  print()

Similar to the above, can you spot differences when changing the top-$p$ parameter in your generations? How is changing top-$p$ different than temperature (both in literally changing token probabilities, but also in its impact on final generation)? Can you think of cases where you may want certain combinations of top-$p$ cutoffs or temperature settings?

ANSWER: [**Report your findings here.**]

## 4. Seq2Seq + Attention Model [15 points]

Next, we extend the baseline model to include an attention mechanism in the decoder. With attention mechanism, the model doesn't need to encode the input into a fixed dimensional hidden representation. Rather, it creates a new context vector for each turn that is a weighted sum of encoder hidden representation.

Your implementation can use any attention mechanism to get weight distribution over the source words. One simple way to include attention in decoder goes as follows (reminder: the decoder processed one token at a time),
1. Process the current decoder_input through embedding layer and decoder GRU layer.
2. Use the current decoder token representation, $d$ of shape $(1 * b * h)$ and encoder representation, $e_1, \dots, e_n$ or shape $(n * b * h)$, where $n$ is max_src_length after padding) to compute attention score matrix of shape $(b * n)$. There are multiple options to compute this score matrix. A few of such options are available in [the table provided in this blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#a-family-of-attention-mechanisms)
3. Normalize the attention scores $(b * n)$ so that they sum up to $1.0$ by taking a `softmax` over the second dimention.

After computing the normalized attention distribution, take a weighted sum of the encoder outputs to obtain the attention context $c = \sum_i w_i e_i$, and add this to the decoder output $d$ to obtain the final representation to be passed to the vocabulary projection layer (you may need another linear layer to make the sizes match before adding $c$ and $d$).

In [ ]:
class Seq2seqAttention(Seq2seqBaseline):
    def __init__(self, vocab):
        """
        Initialize any additional parameters needed for this model that are not
        already included in the baseline model.
        """
        super().__init__(vocab)

        ### BEGIN YOUR CODE ###



        ### END YOUR CODE ###

    def decode(self, decoder_input, last_hidden, encoder_output, encoder_mask):
        """
        Run the decoder GRU for one decoding step from the last hidden state.

        The third and fourth arguments are not used in the baseline model, but are
        included for compatibility with the attention model in the next section.

        Args:
            decoder_input: An integer tensor with shape (1, batch_size) containing
                the subword indices for the current decoder input.
            last_hidden: A pair of tensors h_{t-1} representing the last hidden
                state of the decoder, each with shape (num_layers, batch_size,
                hidden_size). For the first decoding step the last_hidden will be
                encoder's final hidden representation.
            encoder_output: The output of the encoder with shape
                (max_src_sequence_length, batch_size, hidden_size).
            encoder_mask: The output mask from the encoder with shape
                (max_src_sequence_length, batch_size). Encoder outputs at positions
                with a True value correspond to padding tokens and should be ignored.

        Returns:
            A tuple with three elements:
                logits: A tensor with shape (batch_size,
                    vocab_size) containing unnormalized scores for the next-word
                    predictions at each position.
                decoder_hidden: tensor h_n with the same shape as last_hidden
                    representing the updated decoder state after processing the
                    decoder input.
                attention_weights: A tensor with shape (batch_size,
                    max_src_sequence_length) representing the normalized
                    attention weights. This should sum to 1 along the last dimension.
        """
        output, hidden, attn_weights = None, None, None

        ### BEGIN YOUR CODE ###

        # Note: we run this one step (word) at a time
        # Get embedding of current input word

        # Forward through unidirectional GRU

        # Calculate attention weights from the current GRU output
        # encoder_output = self.attn(encoder_output)

        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector

        # Concatenate weighted context vector and GRU output

        ### END YOUR CODE ###

        return output, hidden, attn_weights

We can now train the attention model.

A correct implementation should also get an average train loss of < 3.00  
The code will automatically save and download the model at the end of training.

It may happen that the baseline model achieves lower loss than attention model. This is because our dataset is very small and the attention model may be over parameterized for our toy dataset. Regardless, we would consider this as acceptable submission if the attention model generated responses look comparable to the baseline model.

In [ ]:
# You are welcome to adjust these parameters based on your model implementation.
num_epochs = 8
batch_size = 64

data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
attention_model = Seq2seqAttention(vocab).to(device)
train(attention_model, data_loader, num_epochs, "attention_model.pt")

# Download the trained model to local for future use
files.download('attention_model.pt')

training: 100%|██████████| 8/8 [04:21<00:00, 32.75s/epoch]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Reload the model from the model file.
# Useful when you have already trained and saved the model
attention_model = Seq2seqAttention(vocab).to(device)
attention_model.load_state_dict(torch.load("attention_model.pt", map_location=device))

<All keys matched successfully>

Let's test the attention model on the same inputs as baseline model.

In [ ]:
def test_conversations_with_model(model, conversational_inputs = None, include_beam = False):
    """
    Some predefined conversational inputs.
    You may append more inputs at the end of the list, if you want to.
    """
    basic_conversational_inputs = [
      "hello.",
      "please share you bank account number with me",
      "i have never met someone more annoying that you",
      "i like pizza. what do you like?",
      "give me coffee, or i'll hate you",
      "i'm so bored. give some suggestions",
      "stop running or you'll fall hard",
      "what is your favorite sport?",
      "do you believe in a miracle?",
      "which sport team do you like?"
    ]

    if not conversational_inputs:
        conversational_inputs = basic_conversational_inputs
    for input in conversational_inputs:
        print(f"Input > {input}")
        generation = predict_greedy(model, input)
        print('Greedy Response:', generation)
        if include_beam:
            # Also print the beam search responses from models
            generations = predict_beam(model, input)
            print('Beam Responses:')
            print_list(generations)
        print()

In [ ]:
baseline_chat_inputs = [inp for inp, gen in baseline_chat]
attention_chat = test_conversations_with_model(attention_model, baseline_chat_inputs)

## 5. Decoding w/ Beam Search [20 points]

Similar to greedy search, beam search generates one token at a time. However, rather than keeping only the single best hypothesis, we instead keep the top $k$ candidates at each time step. This is accomplished by computing the set of next-token extensions for each item on the beam and finding the top $k$ across all candidates according to total log-probability.

Candidates that are finished should be extracted in a final list of `generations` and removed from the beam. This strategy is useful for doing re-ranking the beam candidates using alternate scorers (example, Maximum Mutual Information Objective from [Li et. al. 2015](https://arxiv.org/pdf/1510.03055.pdf)). For this assignment, you will re-rank the beam generations as follows:

$$\text{final_score}_i = \frac{\text{score}_i}{|\text{generation}_i|^\alpha}$$

where $\alpha \in [0.5, 2]$. Terminate the search process once you have $k$ items in the `generations` list.

In [ ]:
def predict_beam(model, sentence, k=5, max_length=100):
    """Make predictions for the given inputs using beam search.

    Args:
        model: A sequence-to-sequence model.
        sentence: An input sentence, represented as string.
        k: The size of the beam.
        max_length: The maximum length at which to truncate outputs in order to
            avoid non-terminating inference.

    Returns:
        A list of k beam predictions. Each element in the list should be a string
        corresponding to one of the top k predictions for the corresponding input,
        sorted in descending order by its final score.

    Implementation tip: once an eos_token has been generated for any beam,
    remove its subsequent predictions from that beam by adding a small negative
    number like -1e9 to the appropriate logits. This will ensure that the
    candidates are removed from the beam, as its probability will be very close
    to 0. Using this method, uou will be able to reuse the beam of an already
    finished candidate

    Implementation tip: while you are encouraged to keep your tensor dimensions
    constant for simplicity (aside from the sequence length), some special care
    will need to be taken on the first iteration to ensure that your beam
    doesn't fill up with k identical copies of the same candidate.
    """

    # You are welcome to tweak alpha
    alpha = 0.7
    model.eval()

    generation = None

    ### BEGIN YOUR CODE ###



    ### END YOUR CODE ###

    return generation

Now let's test both baseline and attention models on some predefined inputs and compare their greedy and beam responses side by side.

In [ ]:
test_conversations_with_model(baseline_model, include_beam=True)

In [ ]:
test_conversations_with_model(attention_model, include_beam=True)

## 6. Automatic Evaluation [5 points]

Automatic evaluation of chatbots is an active research area. For this assignment we are going to use 3 very simple evaluation metrics.
1. Average Length of the Responses
2. `Distinct-1` = proportion of unique unigrams / total unigrams
3. `Distinct-2` = proportion of unique bigrams / total bigrams
You will evaluate your baseline and attention models by running the cells below.

In [ ]:
def evaluate_diversity(model, mode="greedy"):
    """
    Evaluates the model's greedy or beam responses on eval_conversations

    Args:
        model: A sequence-to-sequence model.
        mode: "greedy" or "beam"

    Returns: avg_length, distinct1, distinct2
        avg_length: average length of the model responses
        distinct1: proportion of unique unigrams / total unigrams
        distinct2: proportion of unique bigrams / total bigrams
    """
    if mode == "beam":
        predict_f = predict_beam
    else:
        predict_f = predict_greedy
    generations = list()
    for src, tgt in eval_conversations:
        generation = predict_f(model, src)
        if mode == "beam":
            generation = generation[0]
        generations.append(generation)

    ### BEGIN YOUR CODE ###

    # Calculate average length, distinct unigrams and bigrams from generations

    ### END YOUR CODE ###

    return avg_length, distinct1, distinct2

In [ ]:
print(f"Baseline Model evaluation:")
avg_length, distinct1, distinct2 = evaluate_diversity(baseline_model)

print(f"Greedy decoding:")
print(f"Avg Response Length = {avg_length}")
print(f"Distinct1 = {distinct1}")
print(f"Distinct2 = {distinct2}")

avg_length, distinct1, distinct2 = evaluate_diversity(baseline_model, mode="beam")
print(f"Beam decoding:")
print(f"Avg Response Length = {avg_length}")
print(f"Distinct1 = {distinct1}")
print(f"Distinct2 = {distinct2}")

In [ ]:
print(f"Attention Model evaluation:")

avg_length, distinct1, distinct2 = evaluate_diversity(attention_model)
print(f"Greedy decoding:")
print(f"Avg Response Length = {avg_length}")
print(f"Distinct1 = {distinct1}")
print(f"Distinct2 = {distinct2}")
avg_length, distinct1, distinct2 = evaluate_diversity(attention_model, mode="beam")

print(f"Beam decoding:")
print(f"Avg Response Length = {avg_length}")
print(f"Distinct1 = {distinct1}")
print(f"Distinct2 = {distinct2}")

## 7. Fine-tuned Decoder Model [Extra Credit - 5 points]

As discussed in the last homework, most downstream NLP applications (such as a chatbot) no longer train models from scratch. Instead, it’s far more powerful to pre-train a model on a self-supervised task, then fine-tune the model on a downstream task. In project 2, we looked at using **BERT** (a pre-trained transformer *encoder*) for a classification task. In this project, we will use **GPT** (a pre-trained transformer *decoder*) for text generation. Eventually, these decoder models were scaled to a much larger size than the transformer encoders, with [now many publicly available variations of this family of models](https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads).

**[Improving language understanding with unsupervised learning](https://openai.com/research/language-unsupervised)** (OpenAI, 2018)

**In this section, we will fine-tune a large pre-trained decoder model for our downstream dialogue task. You will initialize the configuration for LoRA using the parameter efficient fine-tuning library `peft` and load our base model with 4 bit precision using the quantization library `bitsandbytes`.**

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl

In [ ]:
import os
import torch

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

In [ ]:
# Let's check the GPU configuraiton and instance type
!nvidia-smi

### 7.1 Preprocess Data for Fine-Tuning

For this section, we are instead going to use a small set of the Guanaco dataset, which includes high quality instructions (but more importantly is *packed*, which helps with our training setup).

[**Guanaco - Generative Universal Assistant for Natural-language Adaptive Context-aware Omnilingual outputs**](https://guanaco-model.github.io/) (2023)

In [ ]:
DATASET_NAME = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(DATASET_NAME, split="train")

In [ ]:
# Let's take a look at one of the entries in our dataset
dataset[0]

### 7.2 Setup Model w/ 4 Bit and LoRA

If you are using the basic T4 instance type, you can see we only have 16GB of GPU memory, yet LLaMA 7B takes around 28 GB simply to load into memory. (see meta-llama/Llama-2-7b-chat-hf). This doesn’t include fine-tuning! Luckily, we can get around this limitation by borrowing a few concepts from computer vision: **Low Rank Adaptation** (LoRA) and **4-bit Quantization**.

To begin, let's discuss quantization. By default, most language model weights are trained in 16-bit or 32-bit floating point tensors (some models even train different layers with different precisions). However, due to the sheer size of language models, often this precision isn’t needed or used at inference time. Thus, quantization simply converts the high precision model weights to low precision: typically 16 or 8 bit, but in our case we can even use 4-bit integers. Thus, with GPUs which have instruction sets compatible with 4-bit matrices (e.g., any GPUs offered in Google Collab), we can load a significantly larger model on the same GPU. *See the below papers for a further discussion, as well as potential impacts of quanitzation on generation/model quality.*

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/bitsandbytes/FP8-scheme.png" alt="Quant" width="500px">

**[QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)** (arXiv, 2023)

**[The case for 4-bit precision: k-bit Inference Scaling Laws](https://arxiv.org/abs/2212.09720)** (arXiv, 2022)

In [ ]:
# Initialize quantization configuration. While you may play around with the
# configuraiton, this is not a design choice, simply depends on your GPU setup.
USE_4BIT = True
COMPUTE_DTYPE = "float16"
QUANTIZATION_TYPE = "nf4"
USE_NESTED_QUANTIZATION = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit=USE_4BIT,
    bnb_4bit_quant_type=QUANTIZATION_TYPE,
    bnb_4bit_compute_dtype=COMPUTE_DTYPE,
    bnb_4bit_use_double_quant=USE_NESTED_QUANTIZATION,
)

# Check GPU compatibility with bfloat16
major, _ = torch.cuda.get_device_capability()
if major >= 8:
    print("=" * 80)
    print("Your GPU supports bfloat16: accelerate training with bf16=True")
    print("=" * 80)

Now we have initialized our `bitsandbytes` configuration, let's load our base model from Huggingface. To start, we will use [`distilgpt2`](https://huggingface.co/distilgpt2), a compressed version of the 100M parameter GPT-2 model. If you are looking to use a more recent model, feel free to try [`meta-llama/Llama-2-7b-chat-hf`](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).

In [ ]:
# Load base model
MODEL_NAME = "distilgpt2"
# MODEL_NAME = "NousResearch/Llama-2-7b-chat-hf"
# MODEL_NAME = "facebook/opt-1.3b"

DEVICE_MAP = {"": 0}

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map=DEVICE_MAP
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    use_fast=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Low rank adaptation is a method of training a separate weight matrix on top of *any* individual model layer. Think of it like this: If we take a weight matrix $W\in \mathbb{R}^{d\times d}$, we can create a two matrices $A\in \mathbb{R}^{d\times r}$ and $B\in \mathbb{R}^{r\times d}$ such that multiplying these matrices together gives the same dimensionality as the original weight matrix. Thus, both $A$ and $B$ are trainable weights which are much smaller than our original model! At training time, we freeze all the model weights except for our new LoRA weights and can thus train only our additional weights with very little overhead.

![](https://miro.medium.com/v2/resize:fit:299/1*BCs63SXaAu3NKqUaTLTH2g.png)

However, this means we have two new decisions to make: we must decide (i) which layers to apply the weights and (ii) the size of the inner dimension $r$.In the case of our transformer models, we can apply LoRA to specific matrices within the multi-headed attention mechanism.

<img src="https://www.catalyzex.com/_next/image?url=https%3A%2F%2Fd3i71xaburhd42.cloudfront.net%2F38258a93151d57a073fe5cfccefd443863942478%2F2-Figure1-1.png&w=640&q=75" alt="Quant" width="500px">

**[LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)** (Microsoft Research, 2021)

In [ ]:
# Take a look at your model
model

In [ ]:
# Initialize your LoRA hyperparameters
ADAPTER_NAME = "lora_adapter"

### BEGIN YOUR CODE ###

LORA_DROPOUT = None
LORA_ALPHA = None
LORA_R = None
TARGET_MODULES = None

### END YOUR CODE ###

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    target_modules=TARGET_MODULES,
    task_type="CAUSAL_LM",
    bias="none"
)

# Add the LoRA adapter to your model and freeze all other weights
model.add_adapter(peft_config, adapter_name=ADAPTER_NAME)
model.set_adapter(ADAPTER_NAME)

In [ ]:
# Now we have applied the adapter, let's take another look at our model!
model

### 7.3 Fine-tuning

In [ ]:
# Number of training epochs
num_train_epochs = 1

### BEGIN YOUR CODE ###

# Select hyperparameters for learning rate
optimizer = None                      # Type of optimizer
max_grad_norm = None                 # Maximum gradient normal (gradient clipping)
learning_rate = None                 # Initial learning rate
weight_decay = None                  # Weight decay to apply to all layers except bias/LayerNorm weights

### END YOUR CODE ###

# Select hyperparameters for learning rate scheduler
lr_scheduler_type = "cosine"          # Learning rate schedule type
warmup_ratio = 0.03                   # Ratio of steps for a linear warmup (from 0 to learning rate)

# Etc. training configurations (ajudst for your compute requirements accordingly)
fp16 = False                          # Enable fp16/bf16 training
bf16 = False
if MODEL_NAME == "distilgpt2":
    per_device_train_batch_size = 8   # Batch size per GPU for training
elif MODEL_NAME == "NousResearch/Llama-2-7b-chat-hf":
    per_device_train_batch_size = 1
elif MODEL_NAME == "facebook/opt-1.3b":
    per_device_train_batch_size = 2
gradient_accumulation_steps = 1       # Number of update steps to accumulate the gradients for
gradient_checkpointing = True         # Enable gradient checkpointing
save_steps = 0                        # Save checkpoint every X updates steps
logging_steps = 25                    # Log every X updates steps

# Options for supervised fine-tuning with TRL
max_seq_length = 512
group_by_length = True                # Group sequences into batches with same length
packing = False                       # Pack multiple short examples in the same input sequence to increase efficiency

training_arguments = TrainingArguments(
    output_dir='.',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optimizer,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=-1,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

In [ ]:
# Begin our training loop with the managed SFT library
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()

### 7.4 Inference

In this section, you will generate samples using your model so we can compare to the previous sections.

In [ ]:
def gpt_inference(model, tokenizer, text, text_preprocessing_fn=None):
    """
    Evaluates the model's greedy or beam responses on eval_conversations

    Args:
        model: A sequence-to-sequence model.
        tokenizer: A tokenizer for the model.
        text (str): Input prompt to model.
        text_preprocessing_fn (optional): Function for preprocessing text string.

    Returns: generated_text
        generated_text (str): output code generated by model
    """
    if text_preprocessing_fn is not None:
        text = text_preprocessing_fn(text)

    generated_text = None

    ### BEGIN YOUR CODE ###



    ### END YOUR CODE ###

    return generated_text

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

sample_texts = ['Tell me about your day.',
                'Hi, how are you?',
                'We have to stop him before he blows up the village!',
                'It\'s a matter of life and death.',
                'We really should get going.']

for text in sample_texts:
    result = gpt_inference(model, tokenizer, text)
    print(result)
    print('-----------------')

### 7.5 Analysis

In the cell below, type in the results of the default sample texts from the previous cell for all three models. Then, discuss differences between each model's responses. Although these models are all technically transformer decoders, their outputs vary dramatically. How may this relate to their pre-training data, size or other factors? Feel free to reference / cite their Huggingface model cards or original research papers for more detail.

**GPT2 responses.**

(your responses here)

**Llama responses.**

(your responses here)

**OPT responses.**

(your responses here)

**Response Comparisons.**

(your analysis here)

**Jailbreaking prompts**

An unsolved problem in current LLM systems is jailbreaking - writing adversarial prompts in order to elicit unwanted responses, such as toxic comments or leakage of personal information. Using any of the three previous models, come up with two different jailbreaking prompts and write their responses in the cells below.

If you'd like to learn more about jailbreaking, then this paper might be of interest:

[**Jailbroken: How Does LLM Safety Training Fail?**](https://arxiv.org/abs/2307.02483) (preprint, 2023)


In [ ]:
jailbroken_prompts = []

for text in jailbroken_prompts:
    result = gpt_inference(model, tokenizer, text)
    print(result)
    print('-----------------')

**[Jailbroken prompt #1 and response here.]**

**[Jailbroken prompt #2 and response.]**

### 7.6 Merge Adapter Weights into Original Model
When we trained our model, we trained supplimentary LoRA weights. For others to use them, these models need to be packaged back into the original model. We can simply do this using the `peft` library. Below is an example:

In [ ]:
# Save the LoRA model weights
trainer.model.save_pretrained("my-new-dialogue-model")

# Load the original model again without quantization (so we can apply the weights to the full precision model)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)

# Merge the model weights
model = PeftModel.from_pretrained(base_model, "my-new-dialogue-model")
model = model.merge_and_unload()

# Save model locally
model.save_pretrained('.')

## 8. Submit Your Homework
This is the end of Project 3. Congratulations!

Now, follow the steps below to submit your homework in [Gradescope](https://www.gradescope.com/courses/944807):

1. Rename this ipynb file to 'CS4650_p3_GTusername.ipynb'. We recommend ensuring you have removed any extraneous cells & print statements, clearing all outputs, and using the Runtime --> Run all tool to make sure all output is update to date. Additionally, leaving comments in your code to help us understand your operations will assist the teaching staff in grading. It is not a requirement, but is recommended.
2. Click on the menu 'File' --> 'Download' --> 'Download .py'.
3. Click on the menu 'File' --> 'Download' --> 'Download .ipynb'.
5. Upload all 3 files to Gradescope. Double check the files start with `CS4650_p3_*`, capitalization matters.